# 📉 Securitization, ABS, and the 2007 Financial Crisis

## Overview
This notebook explores the process of **securitization**, focusing on **Asset-Backed Securities (ABS)** and **Collateralized Debt Obligations (CDOs)** backed by ABS, as discussed in Chapter 8 of John Hull's *Options, Futures, and Other Derivatives*.

## 🏦 What is Securitization?
Securitization is the financial process of pooling various types of debt (like mortgages, auto loans, or credit card debt) and selling these pools as **securities** to investors. It enables financial institutions to move debt off their balance sheets and generate liquidity, but also introduces complexity and layers of risk.

### Key Components:
- **Originators**: Banks or other financial institutions that originate loans (e.g., mortgages).
- **Special Purpose Vehicles (SPVs)**: Entities created to pool the loans and issue securities backed by them.
- **Tranches**: The pooled debt is split into slices called tranches, each offering different levels of risk and return.

## 💼 Asset-Backed Securities (ABS)
An **Asset-Backed Security (ABS)** is a type of securitized financial product backed by a pool of non-mortgage loans, such as auto loans, student loans, or credit card receivables.

### How ABS are Constructed:
1. **Pooling**: A collection of similar types of loans (auto loans, credit card debt, etc.) is gathered by an originator (like a bank).
2. **Securitization**: These pooled loans are sold to an SPV, which packages them into securities.
3. **Tranching**: The ABS is divided into different tranches. Each tranche has a different risk profile:
   - **Senior Tranches**: The safest, with first claim on cash flows from the loan repayments.
   - **Mezzanine Tranches**: Riskier than senior tranches, but offer higher returns.
   - **Equity Tranches**: The riskiest, receiving payments only after the higher-priority tranches are paid.

### Risks in ABS:
- **Credit risk**: The chance that borrowers default on their loans.
- **Prepayment risk**: Borrowers may repay their loans earlier than expected, disrupting the cash flow.

## 🏦 ABS CDO (Collateralized Debt Obligation)
A **Collateralized Debt Obligation (CDO)** is a financial product constructed from a pool of **asset-backed securities (ABS)** or other debt instruments, typically structured into tranches based on risk. CDOs introduce an additional layer of securitization, making them even more complex.

### How ABS CDOs are Constructed:
1. **Collecting ABS**: The SPV purchases a pool of ABS, which themselves are already backed by underlying loans like mortgages or credit card debt.
2. **Pooling the ABS**: The ABS are bundled together to create a diversified pool of assets.
3. **Tranching the CDO**: Similar to ABS, the CDO is divided into tranches based on risk:
   - **Senior Tranches**: First in line for payments; lowest risk but lowest returns.
   - **Mezzanine Tranches**: Intermediate risk and return.
   - **Equity Tranches**: Highest risk, as they absorb losses first, but with the potential for higher returns.
4. **Ratings**: Each tranche is given a credit rating by rating agencies based on its risk. Senior tranches often receive higher credit ratings (e.g., AAA), while equity tranches are riskier and rated lower.

### Risks in ABS CDO:
- **Systemic Risk**: Since ABS CDOs are composed of other securitized products, their risks are linked to the performance of the underlying ABS, which can result in cascading losses.
- **Complexity and Lack of Transparency**: The multiple layers of securitization make it difficult to assess the true risk of CDOs, leading to a false sense of security prior to the crisis.

## 📉 The 2007 Financial Crisis: The Role of ABS and ABS CDOs
The 2007 crisis was driven in part by the collapse of **subprime mortgages** packaged into ABS and CDOs. As housing prices fell, mortgage defaults increased, and the complex structures of ABS and ABS CDOs began to unravel, leading to massive losses for investors.

### Crisis Timeline:
1. **Subprime Lending Boom**: Financial institutions extended mortgages to borrowers with poor credit (subprime) and securitized them into ABS.
2. **Demand for CDOs**: Investors’ demand for higher yields led to the creation of CDOs backed by risky subprime ABS.
3. **Housing Market Crash**: When housing prices started to decline, subprime borrowers defaulted on their mortgages in large numbers.
4. **Contagion**: The defaults affected not only the ABS but also the ABS CDOs, leading to severe losses throughout the financial system.

## 🔄 Lessons Learned
The collapse of ABS and ABS CDOs during the crisis highlighted several important lessons:
- **Over-reliance on Ratings**: Investors heavily relied on credit ratings, which underestimated the risk of lower tranches in CDOs.
- **Systemic Risk**: Complex financial instruments like CDOs linked many parts of the financial system, causing widespread damage when defaults increased.
- **Regulation and Transparency**: The crisis led to the introduction of regulations like the **Dodd-Frank Act** to improve transparency and reduce excessive risk-taking.


In [2]:
import numpy as np
import pandas as pd

In [85]:
# ======= I. Simulating Loan Pool =======
def simulate_loan_pool(num_loans: int=100, principal_lower_bound: float=10000, principal_upper_bound: float=50000, default_prob_lower_bound: float=0.01, default_prob_upper_bound: float=0.1) -> pd.DataFrame:
    """
    Generates a random loan pool.
    
    Args:
        num_loans (int): Number of loans to generate.
    
    Returns:
        loan_pool (pd.DataFrame): A DataFrame containing the loan pool.
    """

    loan_pool = pd.DataFrame({
        'loan_id': np.arange(1, num_loans + 1),
        'principal': np.random.uniform(principal_lower_bound, principal_upper_bound, num_loans),
        'default_prob': np.random.uniform(default_prob_lower_bound, default_prob_upper_bound, num_loans)
    })

    return loan_pool

# ======= II. Simulating Loan Defaults and Recovery =======
def simulate_defaults(loan_pool: pd.DataFrame) -> pd.DataFrame:
    """
    Simulate loan defaults based on default probability.
    Args: 
        loan_pool (pd.DataFrame): DataFrame containing loan information.
    
    Returns:
        defaulted_pool (pd.DataFrame): DataFrame with 'defaulted' column indicating if a loan has defaulted.
    """
    defaulted_pool = loan_pool.copy()
    for index, row in loan_pool.iterrows():
        default = np.random.rand() < row['default_prob']
        defaulted_pool.at[index, 'defaulted'] = default
    
    return defaulted_pool

def simulate_recovery(loan_pool: pd.DataFrame, recovery_rate: float = 0.5) -> pd.DataFrame:
    """
    Simulate loan recovery based on recovery rate.
    Args: 
        loan_pool (pd.DataFrame): DataFrame containing loan information.
        recovery_rate (float): Recovery rate for defaulted loans.
    
    Returns:
        recovered_pool (pd.DataFrame): DataFrame with 'recovered_amount' column indicating the recovered amount.
    """
    recovered_pool = loan_pool.copy()
    recovered_pool['recovered_amount'] = np.where(recovered_pool['defaulted'],
                                                  recovered_pool['principal'] * recovery_rate,
                                                  recovered_pool['principal'])
    
    return recovered_pool

# ======= III. Simulating the ABS =======
def simulate_abs(num_loans: int = 100, recovery_rate: float = 0.5, loan_pool: pd.DataFrame = None):
    """
    Simulate the Asset-Backed Security (ABS) cash flows with a waterfall structure.
    
    Args:
        loan_pool (pd.DataFrame): DataFrame containing loan information.
        num_loans (int): Number of loans in the ABS.
        recovery_rate (float): Recovery rate for defaulted loans.
    
    Returns:
        abs_cash_flows (pd.DataFrame): DataFrame with ABS tranche cash flows under a waterfall structure.
    """
    # ------- 1. Simulate Loan Pool -------
    if loan_pool is None:
        loan_pool = simulate_loan_pool(num_loans)
        # ------- 2. Simulate Defaults and Recovery -------
        defaulted_pool = simulate_defaults(loan_pool)
        recovered_pool = simulate_recovery(defaulted_pool, recovery_rate)
    else:
        recovered_pool = loan_pool.copy()

    # ------- 3. Define ABS Structure -------
    expected_cash_flows = recovered_pool['principal'].sum()
    total_cash_flow = recovered_pool['recovered_amount'].sum()
    
    tranches = {
        'Senior': 0.8,      # Senior tranche gets 80% of expected cash flow
        'Mezzanine': 0.15,  # Mezzanine tranche gets 15%
        'Equity': 0.05      # Equity tranche gets 5%
    }
    
    tranche_targets = {
        'Senior': expected_cash_flows * tranches['Senior'],
        'Mezzanine': expected_cash_flows * tranches['Mezzanine'],
        'Equity': expected_cash_flows * tranches['Equity']
    }

    # ------- 4. Allocate Cash Flows -------
    tranche_cash_flows = {}
    remaining_cash_flow = total_cash_flow
    
    for tranche in ['Senior', 'Mezzanine', 'Equity']:
        tranche_target = tranche_targets[tranche]
        
        if remaining_cash_flow >= tranche_target:
            tranche_cash_flows[tranche] = tranche_target
            remaining_cash_flow -= tranche_target
        else:
            tranche_cash_flows[tranche] = remaining_cash_flow
            remaining_cash_flow = 0
    
    # ------- 5. Create ABS Payoff DataFrame -------
    tranche_structure = pd.DataFrame({
        'Tranche': ['Senior', 'Mezzanine', 'Equity'],
        'Target Payment ($)': [tranche_targets['Senior'], tranche_targets['Mezzanine'], tranche_targets['Equity']],
        'Allocated Cash ($)': [tranche_cash_flows['Senior'], tranche_cash_flows['Mezzanine'], tranche_cash_flows['Equity']]
    })
    
    return tranche_structure


In [62]:
"""Running an ABS simulation with 100 loans and a recovery rate of 50%."""
sim_ABS = simulate_abs(num_loans=100, recovery_rate=0.5)
sim_ABS

,Tranche,Target Payment ($),Allocated Cash ($)
0,Senior,2.404222e+06,2.404222e+06
1,Mezzanine,4.507916e+05,4.507916e+05
2,Equity,1.502639e+05,7.474925e+04


In [63]:
# ======= V. Simulating the ABS CDO =======
def simulate_abs_cdo(num_abs: int = 5, num_loans_per_abs: int = 100, recovery_rate: float = 0.5, loan_pool: pd.DataFrame = None):
    """
    Simulate an ABS CDO by pooling the Mezzanine tranches from multiple ABS structures and applying a waterfall structure.
    
    Args:
        num_abs (int): Number of ABS structures to pool.
        num_loans_per_abs (int): Number of loans in each ABS structure.
        recovery_rate (float): Recovery rate for defaulted loans.
    
    Returns:
        cdo_tranche_structure (pd.DataFrame): DataFrame with the CDO tranche structure and cash flows under a waterfall structure.
    """
    # ======= 0. Initialize total cash flows for the ABS Mezzanine tranche =======
    total_mezzanine_target_cash_flow = 0
    total_mezzanine_cash_flow = 0
    
    # ======= 1. Simulate each ABS structure =======
    for i in range(num_abs):
        if loan_pool is None:
            abs_tranche_structure = simulate_abs(num_loans=num_loans_per_abs, recovery_rate=recovery_rate)
        else:
            abs_tranche_structure = simulate_abs(num_loans=num_loans_per_abs, recovery_rate=recovery_rate, loan_pool=loan_pool)
        
        # Aggregate the Mezzanine cash flows from each ABS
        total_mezzanine_target_cash_flow += abs_tranche_structure.loc[abs_tranche_structure['Tranche'] == 'Mezzanine', 'Target Payment ($)'].values[0]
        total_mezzanine_cash_flow += abs_tranche_structure.loc[abs_tranche_structure['Tranche'] == 'Mezzanine', 'Allocated Cash ($)'].values[0]
        
    # ======= 2. Define the CDO tranches structure =======
    cdo_tranches = {
        'Senior': 0.65,      # CDO Senior tranche gets 65% of the Mezzanine cash flow
        'Mezzanine': 0.25,   # CDO Mezzanine tranche gets 25%
        'Equity': 0.1        # CDO Equity tranche gets 10%
    }
    
    # Define target payments for each CDO tranche based on Mezzanine cash flows
    cdo_tranche_targets = {
        'Senior': total_mezzanine_target_cash_flow * cdo_tranches['Senior'],
        'Mezzanine': total_mezzanine_target_cash_flow * cdo_tranches['Mezzanine'],
        'Equity': total_mezzanine_target_cash_flow * cdo_tranches['Equity']
    }
    
    # ======= 3. Apply waterfall structure to allocate cash flows =======
    remaining_cdo_cash_flow = total_mezzanine_cash_flow
    cdo_tranche_cash_flows = {}

    for tranche in ['Senior', 'Mezzanine', 'Equity']:
        tranche_target = cdo_tranche_targets[tranche]
        
        if remaining_cdo_cash_flow >= tranche_target:
            cdo_tranche_cash_flows[tranche] = tranche_target
            remaining_cdo_cash_flow -= tranche_target
        else:
            cdo_tranche_cash_flows[tranche] = remaining_cdo_cash_flow
            remaining_cdo_cash_flow = 0
    
    # ======= 4. Create the CDO tranche structure DataFrame =======
    cdo_tranche_structure = pd.DataFrame({
        'Tranche': ['Senior', 'Mezzanine', 'Equity'],
        'Target Payment ($)': [cdo_tranche_targets['Senior'], cdo_tranche_targets['Mezzanine'], cdo_tranche_targets['Equity']],
        'Allocated Cash ($)': [cdo_tranche_cash_flows['Senior'], cdo_tranche_cash_flows['Mezzanine'], cdo_tranche_cash_flows['Equity']]
    })
    
    return cdo_tranche_structure


In [65]:
"""Running a simulation of an ABS CDO with 5 ABS structures, each containing 100 loans and a recovery rate of 50%."""
sim_ABS_CDO = simulate_abs_cdo(num_abs=5, num_loans_per_abs=100, recovery_rate=0.5)
sim_ABS_CDO

,Tranche,Target Payment ($),Allocated Cash ($)
0,Senior,1.434745e+06,1.434745e+06
1,Mezzanine,5.518248e+05,5.518248e+05
2,Equity,2.207299e+05,2.207299e+05


---
# ***Problems & Exercises***

In [93]:
# 8.11
"""
To simulate the ABS and ABS CDO, we will first generate a loan pool with a single loan of principal amount 100.
Doing this we'll get the results directly in % for each tranche.
"""
loan_pool = simulate_loan_pool(num_loans=1, principal_lower_bound=100, principal_upper_bound=100)
# Instead of simulating defaults and recovery, we will specify a loss percentage on the principal amount. 
loss_percentage = 0.12
loan_pool['recovered_amount'] = loan_pool['principal'] * (1 - loss_percentage)
loan_pool

# Simulate the associated ABS and ABS CDO
sim_ABS = simulate_abs(loan_pool=loan_pool)
sim_ABS_CDO = simulate_abs_cdo(loan_pool=loan_pool)

# Display the results
print(f"ABS Structure: \n{sim_ABS}\n", f"ABS CDO Structure: \n{sim_ABS_CDO}")

ABS Structure: 
     Tranche  Target Payment ($)  Allocated Cash ($)
0     Senior                80.0                80.0
1  Mezzanine                15.0                 8.0
2     Equity                 5.0                 0.0
 ABS CDO Structure: 
     Tranche  Target Payment ($)  Allocated Cash ($)
0     Senior               48.75                40.0
1  Mezzanine               18.75                 0.0
2     Equity                7.50                 0.0


Now that we have created functions to generate the ABS and CDO, the other exercises require simply to change the value of loss_percentage or to change the values of the tranches in the ABS and ABS CDO functions.

Notes : long an ABS tranche is equivalent to short an out of the money put over a "solvability index" (by putting together multiple loans, it tends to reproduce this kind of index). Each tranche represent a different strike price (the equity tranche has the closest strike to be at the money), so a CDO can be seen simply has a put with a strike closer to being at the money.